#### <center>Intermediate Python and Software Enginnering</center>


## <center>Section 04 - Bash - Solutions</center>


### <center>Innovation Scholars Programme</center>
### <center>King's College London, Medical Research Council and UKRI <center>

### 01 Exercises

This notebook for practicing some bash scripting with Jupyter cells. We use the `%%bash` cell magic to send the cell contents to a bash process and capture the results, but you could instead write your answers to a `.sh` file and run them on the command line, the effect is the same.

The exercises will require you to use commands and arguments you haven't seen. The manual pages for almost all commands are accessible with the `man` command, so if you need to look up information on a program like `ls` the command `man ls` will give you a scrollable page of info. If you get stuck Google brings up answers to virtually any question, typically on Stack Overflow or related sites.

This notebook uses two really useful Jupyter features, `!` lines and `%%bash` cells as described above. Lines in code cells beginning with `!` will be treated like bash commands and will be run when the cell runs, this allows you to mix Python and bash commands in one place. Cells beginning with `%%bash` are treated as bash scripts but be aware that user input can't be done and process control doesn't always behave as expected, it's sometimes best to write to script files and run them separately.

### Exercise 1

One of the magic devices on Unix/Linux systems is `/dev/urandom` which will produce an endless stream of pseudo random bytes when read like a file. There's also `/dev/random` but it blocks so we won't use it here. Construct a script to read 100 bytes from `/dev/urandom` and keep only those which correspond to ASCII lower case letters. Suggested commands: `head`, `tr`.

In [4]:
%%bash


head -c 100 /dev/urandom | tr -dc "a-z"

jetxytvmenzwu

### Exercise 2

Print the list of files in this directory in order of size from smallest to largest. Suggested commands: `du`, `sort`.

In [8]:
%%bash 

du -bs * | sort -n

21	hello.sh
60	01_lecture.css
69	example.sh
3065	01_exercises.ipynb
18993	Bash_Cookbook.ipynb
25107	01_lecture.ipynb


### Exercise 3:

Do the same thing but print only those files over 1kB in size.

In [18]:
%%bash 

du -bs * | sort -n | while read i
do 
    size=$(echo $i |awk '{print $1;}')
    if (($size < 1024))
    then 
        echo $i
    fi
done

21 hello.sh
60 01_lecture.css
69 example.sh


### Exercise 4:

Below is the code for creating a simple Python server. Run this cell to generate the script file in the current directory.

In [30]:
%%writefile hello.py

from http.server import SimpleHTTPRequestHandler
from socketserver import TCPServer
from http import HTTPStatus


class Handler(SimpleHTTPRequestHandler):
    def do_GET(self):
        self.send_response(HTTPStatus.OK)
        self.end_headers()
        self.wfile.write(b'Hello, world!\n')


TCPServer.allow_reuse_address = True  # allow the address to be reused faster if you restart the server
TCPServer(('', 8000), Handler).serve_forever()

Overwriting hello.py


Write a script which starts the server in the background, queries it through the address `localhost:8000` and prints the result, then kills the server. This probably won't work correctly here in a Jupyter cell so a separate `.sh` file is needed. Assuming this is called `exercise4.sh` you can run it in the cell below. Suggested commands: `curl`, `kill`.

In [37]:
%%writefile exercise4.sh

# Write your script in this cell then run it to write to the file exercise4.sh
# You can then run your script on the command line or by running the next cell
# This is needed as process control through Jupyter doesn't work like we want.

python hello.py &
lastpid=$!
sleep 1

curl localhost:8000
kill -9 $lastpid

Overwriting exercise4.sh


In [36]:
# use ! to indicate a single line shell command
! bash exercise4.sh

127.0.0.1 - - [09/Jul/2020 17:25:12] "GET / HTTP/1.1" 200 -
Hello, world!


If you get instances of the server running in the background that you can't kill anymore, use the `ps` command or `top` to see what instances are there. These will tell you what the PID is, which you can then use to kill the process.

### Exercise 5

Define a script which prints a directory tree from some starting directory. This can be implemented as functions for example to keep track of state and where you are in the tree. The displayed tree should indicate where each file/directory exists by giving their names prepended with the number of spaces corresponding to depth, this means files will be one space to the right of the containing directory. This will produce a tree like structure and you can include whatever other ascii art or pretty printing you want to make the tree look readable. 

In [127]:
%%bash 

function print_spaces(){
    for((x=0;x<=$1;x++))
    do 
        echo -n '|'
    done
}

function traverse(){
    # special syntax: keyword local is needed since this function is called recursively and each call needs its own copy
    local depth=$2
    
    print_spaces $depth
    echo "\ $(basename $1)"
    
    for i in $1/*
    do
        if [ -d "$i" ]
        then
            traverse "$i" $((depth+1))
        fi
    done
    
    for i in $1/*
    do
        if [ ! -d "$i" ]
        then
            print_spaces $depth
            echo "- $(basename $i)"
        fi
    done
}

function traverse_tree() { traverse $1 0 ; }

traverse_tree ../04_Testing

|\ 04_Testing
||\ 01_exercise
||- interval.py
||- test_example.py
||- test_interval.py
||\ 01_solution
||- interval.py
||- test_interval.py
||\ 02_exercise
||- board.ipynb
||- test_tictactoe.py
||- tictactoe.py
||\ 02_solution
||- tictactoe.py
|- 01_lecture.ipynb
|- 02_lecture.ipynb
|- sqrt.py
|- testsqrt.py
|- typetest.py
